## Compare PODNODE and DMD NIROM results using pre-computed solutions for the flow around a cylinder example


To run this notebook, download the precomputed PODNODE NIROM solution file ```cylinder_online_node_3850824_6.npz``` and ```cylinder_online_node_3851463_7.npz``` from 
```
https://drive.google.com/drive/folders/19DEWdoS7Fkh-Cwe7Lbq6pdTdE290gYSS?usp=sharing
```
and place them in the ```../data/cylinder``` directory. 

The DMD NIROM solutions can be generated by running the ```DMD_cylinder.ipynb``` notebook.

In [ ]:
## Load modules
%matplotlib inline
import numpy as np
import scipy

import os
import gc
from scipy import interpolate
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, ScalarFormatter, FormatStrFormatter

from matplotlib import animation
matplotlib.rc('animation', html='html5')
from IPython.display import display
import matplotlib.ticker as ticker
from matplotlib import rcParams
from matplotlib.offsetbox import AnchoredText


# Plot parameters
plt.rc('font', family='serif')
plt.rcParams.update({'font.size': 20,
                     'lines.linewidth': 2,
                     'axes.labelsize': 16, # fontsize for x and y labels (was 10)
                     'axes.titlesize': 20,
                     'xtick.labelsize': 16,
                     'ytick.labelsize': 16,
                     'legend.fontsize': 16,
                     'axes.linewidth': 2})

import itertools
colors = itertools.cycle(['tab:blue','tab:orange','tab:green','tab:purple',
                          'tab:brown','tab:olive','tab:cyan','tab:pink','tab:red'])
markers = itertools.cycle(['p','d','o','^','s','x','D','H','v']) #,'*'


base_dir   = os.getcwd()
work_dir   = os.path.join(base_dir,'../examples/')
data_dir   = os.path.join(base_dir,'../data/')
nirom_data_dir = os.path.join(base_dir,'../data') 
node_data_dir = os.path.join(base_dir,'../data/cylinder')
fig_dir    = os.path.join(base_dir,'../figures/podnode')


import pynirom
from pynirom.pod import pod_utils as pod
from pynirom.utils import data_utils as du
from pynirom.node import main as nd
from pynirom.node import plotting as pu
from pynirom.node import node as node


In [ ]:
### ------ Import Snapshot data -------------------
data = np.load(data_dir + 'cylinder_Re100.0_Nn14605_Nt3001.npz')
mesh = np.load(data_dir + 'OF_cylinder_mesh_Nn14605_Ne28624.npz')

print('HFM data has {0} snapshots of dimension {1} for p,u and v, spanning times [{2}, {3}]'.format(
                    data['time'].shape[0],data['p'].shape[0],
                    data['time'][0], data['time'][-1]))


## ------- Prepare training snapshots ----------------
print('\n-------Prepare training and testing data---------')
soln_names = ['p', 'v_x', 'v_y']
nodes = mesh['nodes'];  node_ind = mesh['node_ind']
triangles = mesh['elems']; elem_ind = mesh['elem_ind']
Nn = nodes.shape[0]
snap_start = 1250
T_end = 5.0        ### 5 seconds
snap_incr = 4

snap_train, times_train = du.prepare_data(data, soln_names, start_skip=snap_start, T_end=T_end, incr=snap_incr)
print('Using {0} training snapshots for time interval [{1},{2}] seconds'.format(times_train.shape[0],
                                        times_train[0], times_train[-1]))

## ------- Prepare testing snapshots ----------------
pred_incr = snap_incr -3
snap_pred_true, times_predict = du.prepare_data(data, soln_names, start_skip=snap_start, incr=pred_incr)
print('Using {0} testing snapshots for time interval [{1},{2}] seconds'.format(times_predict.shape[0],
                                        times_predict[0], times_predict[-1]))


## ------- Save full HFM data without spinup time -----
snap_data, times_offline = du.prepare_data(data, soln_names, start_skip=snap_start,)

DT = (times_offline[1:] - times_offline[:-1]).mean()
Nt = times_offline.size
Nt_online = times_predict.size
## Normalize the time axis. Required for DMD fitting
tscale = DT*snap_incr            ### Scaling for DMD ()
times_offline_dmd = times_offline/tscale   ## Snapshots DT = 1
times_online_dmd = times_predict/tscale



del data
del mesh
gc.collect()

In [ ]:
## Load best NODE models

best_models = [x for x in os.listdir(node_data_dir) if not os.path.isdir(os.path.join(node_data_dir, x))]
unode = {}
for nn,file in enumerate(best_models):
    unode[nn] ={}
    print('%d: '%(nn+1)+"Loading NODE model %s"%(file.split('node_')[1].split('.npz')[0]))
    for key in soln_names:
        unode[nn][key] = np.load(os.path.join(node_data_dir,file))[key]
print("Loaded %d NODE models"%(len(best_models)))

In [ ]:
## Load best DMD models

DMD1 = np.load(os.path.join(nirom_data_dir,'cylinder_online_dmd_r20.npz'))
Xdmd1 = DMD1['dmd']; X_true = DMD1['true']; 
DMD2 = np.load(os.path.join(nirom_data_dir,'cylinder_online_dmd_r8.npz'))
Xdmd2 = DMD2['dmd']; 
interleaved_snapshots = DMD1['interleaved'] ## True if snapshots were interleaved while performing DMD

del DMD1
del DMD2
gc.collect()


In [ ]:
## Compute the POD coefficients
# trunc_lvl = 0.9999995
trunc_lvl = 0.99

snap_normalized, snap_mean, U, D, W = pod.compute_pod_multicomponent(snap_train)
nw, U_r = pod.compute_trunc_basis(D, U, eng_cap = trunc_lvl)
Z_train = pod.project_onto_basis(snap_train, U_r, snap_mean)


In [ ]:
### ------ Compute the POD coefficients for the truth snapshots on the prediction interval------------------

Z_pred_true = pod.project_onto_basis(snap_pred_true, U_r, snap_mean)
npod_total = np.sum(list(nw.values()))
  
    
true_pred_state_array = np.zeros((times_predict.size,npod_total));
ctr=0
for key in soln_names:
    true_pred_state_array[:,ctr:ctr+nw[key]]=Z_pred_true[key].T
    ctr+=nw[key]

In [ ]:
tmp = {};znode ={}
for nn in range(len(best_models)):
    tmp[nn] = pod.project_onto_basis(unode[nn],U_r,snap_mean)
    znode[nn]=np.zeros((times_predict.size,npod_total))
    ctr=0
    for key in soln_names:
        znode[nn][:,ctr:ctr+nw[key]]=tmp[nn][key].T
        ctr+=nw[key]

In [ ]:
def var_string(ky):
    md = ky
    return md

In [ ]:
### --- Visualize POD coefficients of true solution and NODE predictions

mode1=0; mode2=2; mode3=4
x_inx = times_online_dmd*tscale
start_trunc = 10+0*np.searchsorted(times_predict,times_train[-1])//10
end_trunc = 10*np.searchsorted(times_predict,times_train[-1])//10
end_trunc = end_trunc + (Nt_online - end_trunc)//1
tr_mark = np.searchsorted(times_predict, times_train[-1])
time_ind = np.searchsorted(times_offline, times_predict)
ky1 = 'p'; ky2 = 'v_x'; ky3 = 'v_y'
md1 = var_string(ky1); md2 = var_string(ky2); md3 = var_string(ky3)


fig = plt.figure(figsize=(16,10))
ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,mode1], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax1.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,mode1], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%nn,lw=2,markevery=100)
# lg=plt.legend(ncol=1,fancybox=True,loc='best')
ax1.set_title('$\mathbf{%s}$: $%d$'%(md1,mode1+1))

ax2 = fig.add_subplot(2, 3, 2)
ax2.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,mode2], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax2.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,mode2], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%nn,lw=2,markevery=100)
# lg=plt.legend(ncol=1,fancybox=True,loc='best')
ax2.set_title('$\mathbf{%s}$: $%d$'%(md1,mode2+1))



ax3 = fig.add_subplot(2, 3, 3)
ax3.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,mode3], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax3.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,mode3], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%nn,lw=2,markevery=100)
# lg=plt.legend(ncol=1,fancybox=True,loc='best')
ax3.set_title('$\mathbf{%s}$: $%d$'%(md1,mode3+1))


ax4 = fig.add_subplot(2, 3, 4)
ax4.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,nw['p']+mode1], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax4.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,nw['p']+mode1], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%nn,lw=2,markevery=100)
# lg=plt.legend(ncol=1,fancybox=True,loc='best')
ax4.set_title('$\mathbf{%s}$: $%d$'%(md2,mode1+1))
ax4.set_xlabel('Time (seconds)')


ax5 = fig.add_subplot(2, 3, 5)
ax5.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,nw['p']+mode2], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax5.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,nw['p']+mode2], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%nn,lw=2,markevery=100)
# lg=plt.legend(ncol=1,fancybox=True,loc='best')
ax5.set_title('$\mathbf{%s}$: $%d$'%(md2,mode2+1))
ax5.set_xlabel('Time (seconds)')

ax6 = fig.add_subplot(2, 3, 6)
ax6.plot(x_inx[start_trunc:end_trunc], true_pred_state_array[start_trunc:end_trunc,nw['p']+mode3], color=next(colors), 
         marker=next(markers), markersize=8,label='True',lw=2,markevery=100)
for nn in range(len(best_models)):
    ax6.plot(x_inx[start_trunc:end_trunc], znode[nn][start_trunc:end_trunc,nw['p']+mode3], color=next(colors), 
         marker=next(markers), markersize=8,label='NODE%d'%(nn+1),lw=2,markevery=100)
lg=plt.legend(ncol=5,fancybox=True,bbox_to_anchor=(0.65, -0.18)) #loc='best')
ax6.set_title('$\mathbf{%s}$: $%d$'%(md2,mode3+1))
ax6.set_xlabel('Time (seconds)')

# os.chdir(fig_dir)
# plt.savefig('OF_node_comp_proj_tskip%d_oskip%d.png'%(snap_incr,pred_incr),dpi=300,bbox_extra_artists=(lg,), bbox_inches='tight')

In [ ]:
### Compute spatial RMS errors

fig = plt.figure(figsize=(16,4))
x_inx = times_online_dmd*tscale
start_trunc = 10+0*np.searchsorted(times_predict,times_train[-1])//10
end_trunc = 10*np.searchsorted(times_predict,times_train[-1])//10
end_trunc = end_trunc + (Nt_online - end_trunc)//1
tr_mark = np.searchsorted(times_predict, times_train[-1])
time_ind = np.searchsorted(times_offline, times_predict)
ky1 = 'p'; ky2 = 'v_x'; ky3 = 'v_y'
md1 = var_string(ky1); md2 = var_string(ky2); md3 = var_string(ky3)
Nc = len(soln_names)

dmd_err1 = {}; dmd_err2 = {}
node_err = {};

for nn in range(len(best_models)):
    node_err[nn] = {}
    for ivar,key in enumerate(soln_names):
        node_err[nn][key] = np.linalg.norm(snap_data[key][:,time_ind]- unode[nn][key][:,:], axis=0)/np.sqrt(Nn)   #\

for ivar,key in enumerate(soln_names):
    dmd_err1[key] = np.linalg.norm(X_true[ivar::Nc,:] - Xdmd1[ivar::Nc,:], axis = 0)/np.sqrt(Nn) 
    dmd_err2[key] = np.linalg.norm(X_true[ivar::Nc,:] - Xdmd2[ivar::Nc,:], axis = 0)/np.sqrt(Nn) 

ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(x_inx[start_trunc:end_trunc], dmd_err1[ky1][start_trunc:end_trunc], color=next(colors), 
         marker=next(markers), markersize=8,label='DMD(20):$\mathbf{%s}$'%(md1),lw=2,markevery=100)
ax1.plot(x_inx[start_trunc:end_trunc], dmd_err2[ky1][start_trunc:end_trunc], color=next(colors), 
         marker=next(markers), markersize=8,label='DMD(8):$\mathbf{%s}$'%(md1),lw=2,markevery=100)

for nn in range(len(best_models)):
    ax1.plot(x_inx[start_trunc:end_trunc], node_err[nn][ky1][start_trunc:end_trunc], color=next(colors), 
        markersize=8,marker = next(markers),label='NODE%d:$\mathbf{%s}$'%(nn+1,md1),lw=2,markevery=100)
ymax_ax1 = dmd_err2[ky1][start_trunc:end_trunc].max()
ax1.vlines(x_inx[tr_mark], 0, ymax_ax1, colors ='k', linestyles='dashdot')
ax1.set_xlabel('Time (seconds)');lg=plt.legend(ncol=2,fancybox=True,loc='best')

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(x_inx[start_trunc:end_trunc], dmd_err1[ky2][start_trunc:end_trunc], color=next(colors), 
        markersize=8,marker = next(markers),label='DMD(20):$\mathbf{%s}$'%(md2), lw=2,markevery=100)
ax2.plot(x_inx[start_trunc:end_trunc], dmd_err2[ky2][start_trunc:end_trunc], color=next(colors), 
        markersize=8,marker = next(markers),label='DMD(8):$\mathbf{%s}$'%(md2), lw=2,markevery=100)

for nn in range(len(best_models)):
    ax2.plot(x_inx[start_trunc:end_trunc], node_err[nn][ky3][start_trunc:end_trunc],color=next(colors), 
        markersize=8,marker = next(markers),label='NODE%d:$\mathbf{%s}$'%(nn+1,md2), lw=2,markevery=100)


ymax_ax2 = np.maximum(dmd_err2[ky2][start_trunc:end_trunc].max(), dmd_err2[ky3][start_trunc:end_trunc].max())
ax2.vlines(x_inx[tr_mark],0,ymax_ax2, colors = 'k', linestyles ='dashdot')
ax2.set_xlabel('Time (seconds)');lg=plt.legend(ncol=2,fancybox=True,loc='best')

fig.suptitle('Spatial RMS errors of NIROM solutions', fontsize=18)

# os.chdir(fig_dir)
# plt.savefig('cyl_node_comp_rms_tskip%d_oskip%d.pdf'%(snap_incr,pred_incr), bbox_inches='tight')


In [ ]:
def plot_nirom_soln(Xtrue, Xdmd, Xnode, Nc, Nt_plot, nodes, elems, times_online, comp_names, seed =100, flag = True): 
    
    np.random.seed(seed)
    itime = np.searchsorted(times_online,2.90) #np.random.randint(0,Nt_plot)
    ivar  = 1 #np.random.randint(1,Nc)
    ky = comp_names[ivar]
    tn   = times_online[itime]

    if flag:     ### for interleaved snapshots
        tmp_dmd      = Xdmd1[ivar::Nc,itime]
        tmp_true = Xtrue[ivar::Nc,itime]
    else:
        tmp_dmd      = Xdmd1[ivar*Nn:(ivar+1)*Nn,itime]
        tmp_true = Xtrue[ivar*Nn:(ivar+1)*Nn,itime]
    
    tmp_node = Xnode[ky][:,itime]
    
    fig  = plt.figure(figsize=(18,25));
    ax1   = fig.add_subplot(5, 1, 1)
    surf1 = ax1.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_dmd, cmap=plt.cm.jet)
    ax1.set_title('DMD solution: {0} at t={1:1.2f} seconds, {0} range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                            tmp_dmd.min(),tmp_dmd.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf1, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)
         
    ax2   = fig.add_subplot(5, 1, 3)
    surf2 = ax2.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_true, cmap=plt.cm.jet)
    ax2.set_title('HFM solution: {0} at t={1:1.2f} seconds, {0} range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                        tmp_true.min(),tmp_true.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf2, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)

    err_dmd = tmp_dmd-tmp_true
    ax3   = fig.add_subplot(5, 1, 4)
    surf3 = ax3.tripcolor(nodes[:,0], nodes[:,1],elems, err_dmd, cmap=plt.cm.Spectral)
    ax3.set_title('DMD error: {0} at t={1:1.2f} seconds, error range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                        err_dmd.min(),err_dmd.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf3,orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)
    
    ax4   = fig.add_subplot(5, 1, 2)
    surf4 = ax4.tripcolor(nodes[:,0], nodes[:,1],elems, tmp_node, cmap=plt.cm.jet)
    ax4.set_title('PODNODE solution: {0} at t={1:1.2f} seconds, {0} range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                            tmp_node.min(),tmp_node.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf4, orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)
    
    err_node = tmp_node-tmp_true
    ax5   = fig.add_subplot(5, 1, 5)
    surf5 = ax5.tripcolor(nodes[:,0], nodes[:,1],elems, err_node, cmap=plt.cm.Spectral)
    ax5.set_title('PODNODE error: {0} at t={1:1.2f} seconds, error range = [{2:5.3g},{3:4.2g}]'.format(ky,tn,
                                                            err_node.min(),err_node.max()),fontsize=16)
    plt.axis('off')
    plt.colorbar(surf5,orientation='horizontal',shrink=0.6,aspect=40, pad = 0.03)
    
    return tn

In [ ]:
Nt_plot = np.searchsorted(times_predict, times_train[-1])
itime = plot_nirom_soln(X_true, Xdmd1, unode[0],Nc, Nt_plot, nodes, triangles, times_predict, 
                        soln_names, seed=1990,flag = True)

# os.chdir(fig_dir)
# plt.savefig('cyl_nirom_t%.3f_tskip%d_oskip%d.pdf'%(itime,snap_incr,pred_incr), bbox_inches='tight')
